[View in Colaboratory](https://colab.research.google.com/github/tzhongyan/data-ana-a3/blob/master/a2.ipynb)

# Preparation
Before we start, it is required to have the software as listed below be installed.
- Python 3.5 and above
- A browser with ability to prop console.

## Running on Colabotory
Since google colab treats the python environment seperately, it is required for Google Drive to be mounted onto the filesystem that it is in. We need this because we store the links in JSON format and needed it to be carried around

Run the following codes and follow the instructions to mount google drive into the filesystem. *(You need to run this only once in a notebook per runtime)*



In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

You can check if it is properly mounted.

In [0]:
!ls drive

## Working directory setting
If you are running this notebook on your own computer, you should use empty string. If you are running this notebook on Google Colaboratory, you should edit the following part to `drive/PATH_TO_YOUR_APP_FROM_GDRIVE_ROOT`


In [0]:
# current working directory
# pwd = '' # for local
pwd = 'drive/a2_data_ana/' # our case, change the directory to the correct path

# Obtaining Links for Data Collection
Using the link suggested by class, we had decided to use [EdgeProp](https://www.edgeprop.my)'s data that are available on their website.

Before we start, it is required to obtain the links to the pages that contains data that we need. A simple JavaScript is written to assist the job.

Going to one of the transaction search on EdgeProp (such as [this](https://www.edgeprop.my/transaction/kuala-lumpur?start=2012-01-01&end=2018-07-31&price=0)), and open the console menu.

![Opens console menu. Use img/ss1.png if no internet or broken.](http://tzhongyan.com/img/ss1.png)

We can ignore the warnings shown. Copy the following JavaScript into the console and hit enter.
```js
let t = document.getElementsByClassName('transaction-row');
let ar = [];

for (let i=1; i<t.length; i++) {
    let k = t[i];
    let name = k.childNodes[1].childNodes[1].childNodes[1].firstChild.data;
    let url = k.childNodes[3].childNodes[1].childNodes[1].href;
    ar.push({'neighbourhood':name.trim(), 'url': url});
}
```

![Copy JavaScript and hit Enter. Use img/ss2.png if broken](http://tzhongyan.com/img/ss2.png)

Since the JavaScript buffers the data in `ar` object, we should type in `ar` into the console and hit Enter.
![Type ar and hit enter. Use img/ss1.png if no internet or broken.](http://tzhongyan.com/img/ss3.png)


Right click the object and copy the object.
![Right click to copy the object. Use img/ss1.png if no internet or broken.](http://tzhongyan.com/img/ss4.png)

Open your favourite text editor, and paste using CTRL + V or COMMAND + V. Save the file with `lists.json` or any name of your liking. The file will be used for next step.

# Data Collection
For this assignment, as we are required to do prediction within Klang Valley, data from Kuala Lumpur and Selangor are required to be obtained.

For this case, we have done the link obtains for both Kuala Lumpur and Selangor, and saved the json objects in `list` folder as `kl.json` and `selangor.json` respectively. So the directory should be something like this:
```
 /
 |- list/
    |- kl.json
    |- selangor.json
     
 |- img/
    |- ss1.png
    |- ss2.png
    |- ss3.ong
    |- ss4.pmg
    
 |- a2.ipynb
 
 |- output/ (empty folder now)
```

Next, we will need to crawl the data from the URLs that are stated in the JSON objects.

## Imports and Login Sessions

In [0]:
from bs4 import BeautifulSoup
import requests

import json
import pandas as pd

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

import warnings
warnings.filterwarnings("ignore")

Since EdgeProp requires login to show all the data, we will need to handle login session. 

**Change the below email/password authenticate in EdgeProp.** Failling to do so will only get 10 data for each neighbourhood or township.

In [7]:
email = ""
password = ""


session_requests = requests.session()

""" LOGIN AND GET SESSION """
login_url = "https://www.edgeprop.my/user/login"
# Get login csrf token
login_page = session_requests.get(login_url)
soup = BeautifulSoup(login_page.text, 'html.parser')
authenticity_token = soup.find("input", { "name": "form_build_id" } )["value"]
payload = { "name": email, "pass": password, "form_build_id": authenticity_token, "form_id": "user_login" }
# Perform login
result = session_requests.post(login_url, data = payload, headers = dict(referer = login_url))
""" END LOGIN AND GET SESSION """

' END LOGIN AND GET SESSION '

Define a scrapper function that obtain data from the url upon calling, and return a dataframe with its relative data.

In [0]:
def scrap(url_k):
	page = session_requests.get(url_k, headers = dict(referer = 'https://www.edgeprop.my/transaction/selangor?start=2012-01-01&end=2018-06-31&price=0'))
	soup = BeautifulSoup(page.text, 'html.parser')

	# Extract result
	date_list = soup.find_all("div", class_="transaction-item transaction-date")
	price_list = soup.find_all("div", class_="transaction-item transaction-price")
	type_list = soup.find_all("div", class_="transaction-item transaction-type2")
	sf_list = soup.find_all("div", class_="transaction-item transaction-sqft")
	addr_list = soup.find_all("div", class_="transaction-item transaction-address")

	d = {
		'date': [date.get_text() for date in date_list],
		'price': [locale.atoi(price.get_text()) for price in price_list],
		'house_type': [hse.get_text() for hse in type_list],
		'area_sf': [locale.atoi(area.get_text()) for area in sf_list],
		'addr': [addr.get_text() for addr in addr_list]
	}

	return pd.DataFrame(data=d)

## Crawler
### Selangor Data
Open the json file that you want scrawl, we will be running selangor first:

In [0]:

# load links from json
with open(pwd + 'list/selangor.json') as f:
    data = json.load(f)

Obtaining the URL list, getting the area name from url, and remove the links of areas that are not in klang valley.

In [10]:
# spliting area using the url given
url_list = {
    'area': [d['url'].split('/')[5] for d in data],
    'url': [d['url'] for d in data],
    'neighbourhood': [d['neighbourhood'] for d in data]
}
df_url = pd.DataFrame(data=url_list)

# remove regions listen but not in klang valley
not_in_klang_valley = ['tanjong-sepat', 'tanjong-karang', 'batang-berjuntai', 'jenjarom'
					, 'kerling', 'sepang', 'semenyih', 'batang-kali', 'banting'
					, 'pulau-indah-(pulau-lumut)', 'sungai-besar', 'telok-panglima-garang'	
					, 'serendah', 'rasa', 'kuala-selangor', 'sabak-bernam', 'batu-arang'
					, 'beranang', 'jeram', 'kuala-kubu-baru', 'kuala-selangor', 'kuang'
					, 'sekinchan', 'sungai-pelek']
df_url = df_url[~df_url['area'].isin(not_in_klang_valley)]

df_url.head()

,area,neighbourhood,url
0,klang,Taman Sentosa,https://www.edgeprop.my/transaction/selangor/k...
1,hulu-selangor,Bukit Sentosa,https://www.edgeprop.my/transaction/selangor/h...
2,hulu-selangor,Bandar Bukit Beruntung,https://www.edgeprop.my/transaction/selangor/h...
3,bangi,Bandar Bukit Mahkota Bangi,https://www.edgeprop.my/transaction/selangor/b...
4,cheras-south,Bandar Mahkota Cheras,https://www.edgeprop.my/transaction/selangor/c...


Obtaining the data from URL listed above one by one. 
** *Note: this might take a while, easily 30 mins and beyond**

In [16]:
df = pd.DataFrame(data=[])

# scraping for each url
for index, row in df_url.iterrows():
	df2=scrap(row['url'])
	df2['area'] = row['area']
	df2['neighbourhood'] = row['neighbourhood']
	print(row['area'], ": ", len(df2.index))
	df = df.append(df2)

print('============================================')
print('total length: ', len(df.index))
print(df.head())

klang :  2190
hulu-selangor :  2124
hulu-selangor :  1766
bangi :  1712
cheras-south :  1672
bandar-puncak-alam :  1607
denai-alam :  1584
bandar-saujana-putra :  1370
rawang :  1303
kepong :  1272
shah-alam :  1250
bandar-botanic_bandar-bukit-tinggi :  1205
damansara-perdana :  1190
shah-alam :  1138
ulu-kelang :  1111
bandar-botanic_bandar-bukit-tinggi :  1102
puchong-south :  986
puncak-jalil :  985
puchong-south :  978
shah-alam :  957
gombak :  870
kota-kemuning :  867
shah-alam :  847
bukit-jelutong :  837
rawang :  836
klang :  826
bandar-sunway :  825
bandar-damai-perdana :  788
puchong :  783
shah-alam :  779
klang :  779
bandar-sunway :  755
bandar-country-homes :  717
klang :  702
shah-alam :  694
sungai-buloh :  687
pandan-indah :  683
klang :  658
kajang :  634
hulu-selangor :  631
ampang :  630
pandan-jaya :  624
seri-kembangan :  611
shah-alam :  605
ampang :  575
hulu-selangor :  573
shah-alam :  547
kajang :  535
rawang :  534
bandar-botanic_bandar-bukit-tinggi :  533


damansara-perdana :  376
puchong :  376
seri-kembangan :  374
shah-alam :  374
puchong :  373
kajang :  370
damansara-damai :  367
klang :  365
bandar-sunway :  365
bukit-rahman-putra :  360
puchong :  358
subang-bestari :  352
klang :  350
rawang :  343
glenmarie :  342
ss2 :  340
ara-damansara :  336
batu-caves :  334
petaling-jaya :  331
damansara-damai :  331
bandar-puncak-alam :  329
bandar-sri-damansara :  326
bangi :  326
subang-jaya :  324
bandar-sungai-long :  324
bandar-sungai-long :  323
batu-9th-cheras :  320
damansara-damai :  318
shah-alam :  315
keramat :  314
tropicana :  312
usj :  311
bandar-kinrara-puchong :  307
usj :  307
bandar-puteri-puchong :  303
shah-alam :  302
seri-kembangan :  302
bandar-puteri-puchong :  301
klang :  300
shah-alam :  299
puchong :  299
batu-caves :  297
bangi :  297
puchong :  293
puchong :  293
selayang :  291
bandar-kinrara-puchong :  290
cheras :  290
kajang :  289
pandan-perdana :  0
shah-alam :  288
puchong-south :  288
puchong :  288

cheras :  234
setia-alam_alam-nusantara :  234
balakong :  232
kepong :  230
shah-alam :  230
bandar-sungai-long :  229
damansara-damai :  229
shah-alam :  229
bandar-kinrara-puchong :  229
puchong-south :  228
petaling-jaya :  228
cheras-south :  228
setia-alam_alam-nusantara :  227
bukit-subang :  227
kelana-jaya :  226
puchong :  226
petaling-jaya :  223
pandan-jaya :  223
klang :  223
bandar-country-homes :  223
seri-kembangan :  223
ara-damansara :  222
bandar-kinrara-puchong :  222
puchong-south :  221
damansara-damai :  221
batu-caves :  221
ampang :  219
seri-kembangan :  219
ampang :  218
kajang :  218
usj :  218
bandar-kinrara-puchong :  217
seri-kembangan :  217
taman-ttdi-jaya :  216
damansara-perdana :  216
damansara-damai :  216
subang-jaya :  214
bandar-botanic_bandar-bukit-tinggi :  213
rawang :  212
seri-kembangan :  212
petaling-jaya :  212
bangi :  212
port-klang :  211
bangi :  211
puchong :  209
setia-alam_alam-nusantara :  207
bandar-botanic_bandar-bukit-tinggi : 

kajang :  174
kajang :  174
setia-alam_alam-nusantara :  174
batu-9th-cheras :  174
kajang :  174
bandar-kinrara-puchong :  172
bangi :  172
bandar-kinrara-puchong :  172
damansara-damai :  172
bandar-sri-damansara :  172
ara-damansara :  172
sungai-buloh :  171
cheras :  171
sungai-buloh :  170
port-klang :  170
subang-jaya :  170
rawang :  170
putra-heights :  170
bandar-sungai-long :  169
putrajaya :  169
petaling-jaya :  169
tropicana :  169
ampang :  169
klang :  168
subang :  168
ampang :  167
ampang :  167
shah-alam :  167
dengkil :  167
seri-kembangan :  165
bandar-country-homes :  165
damansara-utama :  164
usj :  163
petaling-jaya :  163
damansara-damai :  163
port-klang :  163
petaling-jaya :  163
usj :  162
mutiara-damansara :  162
pandan-indah :  162
damansara-damai :  162
setia-alam_alam-nusantara :  162
batu-caves :  162
kajang :  161
puchong-south :  161
klang :  161
kepong :  161
shah-alam :  161
keramat :  159
seri-kembangan :  159
damansara-perdana :  159
setia-alam_

bangi :  145
klang :  145
pandan-indah :  145
balakong :  145
bandar-country-homes :  145
ara-damansara :  145
rawang :  145
shah-alam :  144
shah-alam :  144
bandar-sunway :  144
puchong :  144
kapar :  143
ampang :  143
petaling-jaya :  143
seri-kembangan :  143
usj :  142
kelana-jaya :  142
cyberjaya :  142
petaling-jaya :  142
kota-damansara :  142
usj :  141
bandar-kinrara-puchong :  141
kajang :  140
kota-damansara :  140
kajang :  139
bandar-sungai-long :  139
batu-9th-cheras :  139
kayu-ara :  139
ampang :  138
usj :  138
cyberjaya :  138
kajang :  137
puchong :  137
selayang :  137
setia-alam_alam-nusantara :  136
cheras-south :  136
klang :  136
usj :  135
batu-9th-cheras :  135
bandar-sungai-long :  135
bandar-botanic_bandar-bukit-tinggi :  135
kajang :  135
shah-alam :  135
seri-kembangan :  135
hulu-selangor :  134
bangi :  134
kajang :  133
gombak :  133
klang :  133
petaling-jaya :  133
subang-jaya :  133
bandar-sunway :  132
bandar-kinrara-puchong :  132
kapar :  131
am

...

rawang :  14
kota-kemuning :  14
klang :  13
klang :  13
klang :  13
klang :  13
cheras-south :  13
rawang :  13
kajang :  13
batu-caves :  13
cheras-south :  13
klang :  13
rawang :  13
petaling-jaya :  13
balakong :  13
bangi :  13
bandar-sungai-long :  13
shah-alam :  13
rawang :  13
puchong :  13
seri-kembangan :  13
selayang :  13
puchong-south :  13
seri-kembangan :  13
puchong :  13
cyberjaya :  13
batu-9th-cheras :  13
kapar :  13
sungai-buloh :  13
bandar-kinrara-puchong :  13
subang-jaya :  13
kapar :  13
shah-alam :  13
setia-alam_alam-nusantara :  13
sungai-buloh :  13
seri-kembangan :  13
klang :  12
kayu-ara :  12
kapar :  12
port-klang :  12
kajang :  12
ampang :  12
kajang :  12
kajang :  12
rawang :  12
subang :  12
balakong :  12
klang :  12
dengkil :  12
ampang :  12
klang :  12
bandar-sungai-long :  12
bandar-sungai-long :  12
subang :  12
hulu-selangor :  12
batu-caves :  12
kayu-ara :  12
port-klang :  12
shah-alam :  12
shah-alam :  12
petaling-jaya :  12
taman-m

kajang :  11
klang :  10
taman-melawati :  10
usj :  10
ampang :  10
ampang :  10
kajang :  10
hulu-selangor :  10
seri-kembangan :  10
hulu-langat :  10
shah-alam :  10
taman-melawati :  10
cheras :  10
cheras-south :  10
klang :  10
cheras :  10
gombak :  10
klang :  10
usj :  10
ampang :  10
klang :  10
batu-9th-cheras :  10
ampang :  10
seri-kembangan :  10
ampang :  10
hulu-langat :  10
shah-alam :  10
kajang :  10
subang :  10
bangi :  10
balakong :  10
cyberjaya :  10
dengkil :  10
bangi :  10
selayang :  10
kapar :  10
rawang :  10
petaling-jaya :  10
gombak :  10
kota-damansara :  10
klang :  10
bandar-kinrara-puchong :  10
shah-alam :  10
klang :  10
rawang :  10
puchong :  10
ampang :  10
damansara-perdana :  10
rawang :  10
ara-damansara :  9
kajang :  9
klang :  9
taman-melawati :  9
ara-damansara :  9
bandar-sungai-long :  9
klang :  9
selayang :  9
bangi :  9
ara-damansara :  9
kayu-ara :  9
klang :  9
bangi :  9
bandar-sungai-long :  9
shah-alam :  9
klang :  9
selayang

rawang :  8
kajang :  8
kapar :  8
serdang :  8
ampang :  0
cyberjaya :  8
sungai-buloh :  7
klang :  7
rawang :  7
dengkil :  7
hulu-langat :  7
bandar-kinrara-puchong :  7
bangi :  7
batu-caves :  7
seri-kembangan :  7
port-klang :  7
shah-alam :  7
balakong :  7
sungai-buloh :  7
kayu-ara :  7
taman-melawati :  7
selayang :  7
batu-caves :  7
seri-kembangan :  7
subang :  7
batu-9th-cheras :  7
kota-kemuning :  7
shah-alam :  7
shah-alam :  7
kota-damansara :  7
shah-alam :  7
puchong :  7
cyberjaya :  7
kajang :  7
klang :  7
kajang :  7
rawang :  7
hulu-langat :  7
bangi :  7
batu-9th-cheras :  7
klang :  7
shah-alam :  7
cyberjaya :  7
batu-9th-cheras :  7
cyberjaya :  7
sungai-buloh :  7
puchong :  6
bandar-country-homes :  6
cyberjaya :  6
gombak :  6
shah-alam :  6
kajang :  6
gombak :  6
country-heights :  6
bangi :  6
usj :  6
port-klang :  6
ulu-kelang :  6
klang :  6
shah-alam :  6
bandar-sungai-long :  6
petaling-jaya :  6
klang :  6
hulu-langat :  6
batu-9th-cheras :  6


taman-melawati :  5
klang :  5
subang :  5
gombak :  5
rawang :  5
cyberjaya :  5
shah-alam :  5
gombak :  5
puchong :  5
bangi :  5
klang :  5
bandar-puteri-puchong :  5
hulu-langat :  5
kayu-ara :  5
petaling-jaya :  5
dengkil :  5
ampang :  5
cyberjaya :  5
hulu-langat :  5
ara-damansara :  5
cyberjaya :  5
rawang :  5
kota-damansara :  5
kajang :  4
klang :  4
shah-alam :  4
kajang :  4
shah-alam :  4
ampang :  4
shah-alam :  4
batu-caves :  4
selayang :  4
puchong :  4
rawang :  4
hulu-selangor :  4
cheras :  4
bandar-sungai-long :  4
puchong :  4
kayu-ara :  4
pandan-jaya :  4
puchong-south :  4
petaling-jaya :  4
klang :  4
usj :  4
seri-kembangan :  4
bangi :  4
bangi :  4
bangi :  4
klang :  4
bangi :  4
rawang :  4
klang :  4
dengkil :  4
shah-alam :  4
bandar-saujana-putra :  4
glenmarie :  4
cyberjaya :  4
selayang :  4
mutiara-damansara :  4
dengkil :  3
kepong :  3
kajang :  3
rawang :  3
cheras-south :  3
ulu-kelang :  3
klang :  3
klang :  3
hulu-selangor :  3
ulu-kelan

hulu-selangor :  2
bandar-kinrara-puchong :  2
klang :  2
puchong-south :  2
kajang :  2
klang :  2
sungai-buloh :  2
damansara-jaya :  2
gombak :  2
petaling-jaya :  2
dengkil :  2
klang :  2
klang :  2
rawang :  2
gombak :  2
bukit-rahman-putra :  2
shah-alam :  2
bangi :  2
setia-alam_alam-nusantara :  2
cyberjaya :  2
petaling-jaya :  2
shah-alam :  2
setia-alam_alam-nusantara :  1
klang :  1
puchong-south :  1
kota-damansara :  1
petaling-jaya :  1
kajang :  1
puchong :  1
seri-kembangan :  1
puchong :  1
subang :  1
cheras :  1
batu-9th-cheras :  1
ampang :  1
klang :  1
petaling-jaya :  1
serdang :  1
subang-jaya :  1
kapar :  1
klang :  1
dengkil :  1
subang :  1
petaling-jaya :  1
selayang :  1
puchong :  1
puchong-south :  1
selayang :  1
denai-alam :  1
setia-alam_alam-nusantara :  1
dengkil :  1
puchong :  1
seri-kembangan :  1
seri-kembangan :  1
klang :  1
damansara-damai :  1
cyberjaya :  1
damansara-damai :  1
selayang :  1
setia-alam_alam-nusantara :  1
shah-alam :  1


And save the result:

In [0]:
# output
df.to_csv(pwd + 'output/selangor.csv', sep=';', index=False )

Note we are using `;` as the csv seperator as using the defaulted `,` will cause problem as the address contains `,`.

### KL data
Basically a repeat of the above, but using json file from KL.

All areas in KL are located in Klang Valley, so removing the items not in Klang Valley is not required.

In [18]:
# load links from json
with open(pwd + 'list/kl.json') as f:
    data = json.load(f)

# spliting area using the url given
url_list = {
    'area': [d['url'].split('/')[5] for d in data],
    'url': [d['url'] for d in data],
    'neighbourhood': [d['neighbourhood'] for d in data]
}
df_url = pd.DataFrame(data=url_list)


df = pd.DataFrame(data=[])
# scraping for each url
for index, row in df_url.iterrows():
	df2=scrap(row['url'])
	df2['area'] = row['area']
	df2['neighbourhood'] = row['neighbourhood']
	print(row['area'], ": ", len(df2.index))
	df = df.append(df2)

print('============================================')
print(df.head())
print('total length: ', len(df.index))

# output
df.to_csv(pwd+'output/kl.csv', sep=';', index=False )

bandar-baru-sri-petaling :  718
taman-setiawangsa :  498
taman-tun-dr-ismail :  486
damansara :  464
jalan-klang-lama-(old-klang-road) :  443
kuchai-lama :  439
salak-selatan :  392
bukit-jalil :  384
bandar-tasik-selatan :  372
dutamas :  362
jalan-klang-lama-(old-klang-road) :  352
kepong :  346
cheras :  342
mont-kiara :  341
pantai-dalam_kerinchi :  340
desa-petaling :  329
wangsa-maju :  328
desa-pandan :  325
setapak :  323
setapak :  312
wangsa-maju :  306
kl-city :  305
dutamas :  294
bandar-menjalara :  294
sungai-besi :  293
setapak :  286
kepong :  281
mont-kiara :  274
taman-oug :  268
titiwangsa :  268
dutamas :  262
taman-desa :  259
bukit-jalil :  259
taman-desa :  0
selayang :  258
cheras :  256
mont-kiara :  247
segambut :  240
salak-selatan :  239
kuchai-lama :  236
bukit-bintang :  235
wangsa-maju :  227
cheras :  227
kepong :  220
kuchai-lama :  219
bandar-menjalara :  210
cheras :  210
wangsa-maju :  208
bukit-jalil :  207
kepong :  207
setapak :  203
cheras :  202

kuchai-lama :  154
setapak :  154
mont-kiara :  153
sentul :  153
klcc :  153
setapak :  152
klcc :  151
klcc :  150
kepong :  150
bukit-prima-pelangi :  149
sungai-besi :  149
setapak :  149
jalan-ipoh :  149
cheras :  148
bukit-jalil :  148
mont-kiara :  147
sentul :  147
bukit-jalil :  147
cheras :  146
mont-kiara :  146
kepong :  146
mont-kiara :  146
sentul :  146
mont-kiara :  146
desa-parkcity :  144
mont-kiara :  144
setapak :  144
sri-hartamas :  144
sentul :  143
sentul :  143
cheras :  143
setapak :  143
sri-hartamas :  143
bukit-jalil :  142
klcc :  142
cheras :  141
bandar-baru-sri-petaling :  141
kl-city :  140
pantai-dalam_kerinchi :  139
kl-city :  139
segambut :  138
bangsar :  137
jalan-ipoh :  136
taman-setiawangsa :  136
kepong :  136
cheras :  136
klcc :  136
kuchai-lama :  136
jalan-klang-lama-(old-klang-road) :  135
damansara :  135
setapak :  134
wangsa-maju :  134
cheras :  133
sunway-spk-damansara :  133
kepong :  132
kepong :  132
bandar-baru-sri-petaling :  

jinjang :  115
mont-kiara :  115
cheras :  113
wangsa-maju :  113
jalan-klang-lama-(old-klang-road) :  113
keramat :  112
bukit-prima-pelangi :  112
kuchai-lama :  112
jalan-klang-lama-(old-klang-road) :  112
kl-city :  111
mont-kiara :  111
wangsa-maju :  111
bangsar :  110
jinjang :  110
damansara :  110
jalan-klang-lama-(old-klang-road) :  109
setapak :  109
kampung-sungai-penchala :  109
jalan-ipoh :  109
setapak :  108
jalan-klang-lama-(old-klang-road) :  108
setapak :  108
mont-kiara :  108
cheras :  108
desa-petaling :  107
jalan-ipoh :  107
bangsar-south :  107
sentul :  107
mont-kiara :  106
cheras :  106
kepong :  106
wangsa-maju :  106
kl-city :  106
mont-kiara :  105
sentul :  105
bangsar :  105
kl-city :  105
taman-u-thant :  0
taman-tun-dr-ismail :  105
cheras :  105
mont-kiara :  104
bukit-jalil :  104
mont-kiara :  104
desa-pandan :  103
mont-kiara :  103
jalan-klang-lama-(old-klang-road) :  103
desa-petaling :  102
bukit-jalil :  102
taman-tun-dr-ismail :  102
mont-kia

kl-city :  84
cheras :  84
ampang :  84
bukit-bintang :  84
damansara :  83
kl-city :  83
kepong :  83
salak-selatan :  83
pantai-dalam_kerinchi :  83
mont-kiara :  83
desa-petaling :  82
klcc :  82
bandar-menjalara :  82
taman-desa :  81
setapak :  80
bukit-jalil :  80
pudu :  80
taman-melawati :  80
jalan-ipoh :  80
kepong :  80
pudu :  80
pantai-dalam_kerinchi :  79
taman-melawati :  79
mont-kiara :  79
sentul :  79
cheras :  79
kampung-pandan :  79
cheras :  79
taman-desa :  78
taman-duta :  78
desa-parkcity :  78
jalan-klang-lama-(old-klang-road) :  78
desa-petaling :  77
cheras :  77
dutamas :  77
taman-desa :  76
wangsa-maju :  76
ampang :  76
sentul :  76
mont-kiara :  75
segambut :  75
setapak :  75
cheras :  75
bandar-baru-sri-petaling :  75
dutamas :  74
cheras :  74
setapak :  74
mont-kiara :  73
taman-tun-dr-ismail :  73
pudu :  73
jalan-ipoh :  73
klcc :  73
mont-kiara :  72
kepong :  72
salak-selatan :  72
bukit-bintang :  72
segambut :  71
dutamas :  71
mont-kiara :  71

selayang :  62
sungai-besi :  62
taman-desa :  61
dutamas :  61
jalan-klang-lama-(old-klang-road) :  61
kampung-sungai-penchala :  61
jalan-ipoh :  61
jalan-ipoh :  61
bukit-bintang :  61
bukit-bintang :  60
jalan-klang-lama-(old-klang-road) :  60
taman-desa :  60
seputeh :  60
bukit-jalil :  60
bukit-bintang :  60
kepong :  60
taman-tun-dr-ismail :  60
desa-petaling :  59
bandar-baru-sri-petaling :  59
taman-tun-dr-ismail :  59
cheras :  59
kepong :  59
bukit-jalil :  58
pantai :  58
jalan-ipoh :  58
taman-desa :  58
taman-setiawangsa :  58
cheras :  58
damansara :  58
kuchai-lama :  58
bangsar :  57
cheras :  57
dutamas :  57
mont-kiara :  56
jalan-ipoh :  56
cheras :  56
mid-valley-city :  56
kepong :  56
jalan-klang-lama-(old-klang-road) :  56
setapak :  56
kuchai-lama :  56
sungai-besi :  56
bangsar-south :  56
desa-parkcity :  56
sri-hartamas :  55
setapak :  55
salak-selatan :  55
setapak :  55
bukit-jalil :  55
damansara-heights :  54
dutamas :  54
jalan-klang-lama-(old-klang-r

...

bangsar :  22
wangsa-maju :  22
jalan-ipoh :  22
pudu :  22
klcc :  22
jalan-klang-lama-(old-klang-road) :  22
pantai :  22
setapak :  22
cheras :  22
taman-tun-dr-ismail :  22
jalan-klang-lama-(old-klang-road) :  22
bukit-bintang :  22
bandar-tun-razak :  22
jalan-klang-lama-(old-klang-road) :  22
segambut :  21
klcc :  21
kl-city :  21
sentul :  21
cheras :  21
sentul :  21
damansara-heights :  21
bangsar :  21
pudu :  21
taman-tun-dr-ismail :  21
setapak :  21
cheras :  21
setapak :  21
setapak :  21
bukit-bintang :  21
kuchai-lama :  21
dutamas :  20
seputeh :  20
jalan-kuching :  20
seputeh :  20
ampang :  20
batu :  20
bangsar :  20
seputeh :  20
cheras :  20
sentul :  20
pantai-dalam_kerinchi :  20
klcc :  20
sentul :  20
bukit-bintang :  20
titiwangsa :  19
bukit-bintang :  19
cheras :  0
klcc :  19
bangsar :  19
seputeh :  19
kl-city :  19
taman-u-thant :  0
selayang :  19
jalan-klang-lama-(old-klang-road) :  19
brickfields :  19
wangsa-maju :  19
taman-setiawangsa :  19
jalan

taman-u-thant :  0
setapak :  16
kenny-hills-(bukit-tunku) :  16
bangsar :  16
sungai-besi :  15
taman-u-thant :  0
cheras :  15
ampang- :  15
bukit-bintang :  15
kuchai-lama :  15
segambut :  15
segambut :  15
kl-city :  15
seputeh :  15
setapak :  15
mont-kiara :  15
taman-desa :  15
jalan-klang-lama-(old-klang-road) :  15
bukit-bintang :  15
setapak :  15
desa-parkcity :  15
jalan-klang-lama-(old-klang-road) :  15
salak-selatan :  15
bangsar :  14
pudu :  14
jalan-klang-lama-(old-klang-road) :  14
sentul :  14
kuchai-lama :  14
jalan-ipoh :  14
sri-hartamas :  14
jalan-ipoh :  14
bangsar :  14
setapak :  14
pantai-dalam_kerinchi :  0
mont-kiara :  14
ampang :  14
damansara-heights :  14
bukit-bintang :  14
cheras :  14
pudu :  14
segambut :  14
jalan-klang-lama-(old-klang-road) :  13
kl-city :  13
setapak :  13
klcc :  13
dutamas :  13
segambut :  13
keramat :  13
salak-selatan :  13
pudu :  13
bangsar :  13
sentul :  13
taman-oug :  13
bukit-jalil :  13
dutamas :  13
bukit-bintang 

jalan-klang-lama-(old-klang-road) :  10
kl-city :  10
jalan-klang-lama-(old-klang-road) :  10
pudu :  10
sri-hartamas :  10
ampang-hilir :  10
taman-tun-dr-ismail :  10
segambut :  10
cheras :  10
segambut :  10
segambut :  10
segambut :  10
setapak :  10
titiwangsa :  10
cheras :  10
cheras :  10
bangsar :  10
klcc :  10
kl-city :  10
mont-kiara :  10
setapak :  10
jalan-klang-lama-(old-klang-road) :  10
bangsar :  9
kl-city :  9
seputeh :  9
kuchai-lama :  9
segambut :  9
mont-kiara :  9
wangsa-maju :  9
cheras :  9
cheras :  9
cheras :  9
cheras :  9
jalan-klang-lama-(old-klang-road) :  9
bangsar :  9
cheras :  9
setapak :  9
bangsar :  8
seputeh :  8
dutamas :  8
setapak :  8
jalan-ipoh :  8
taman-u-thant :  0
damansara :  8
bukit-bintang :  8
bukit-jalil :  8
taman-u-thant :  0
wangsa-maju :  8
klcc :  8
bukit-bintang :  8
pudu :  8
pantai-dalam_kerinchi :  8
damansara :  8
mont-kiara :  8
taman-desa :  8
kl-city :  8
ampang :  8
sentul :  8
pantai-dalam_kerinchi :  8
bangsar :  8

jalan-klang-lama-(old-klang-road) :  6
cheras :  0
cheras :  6
jalan-klang-lama-(old-klang-road) :  6
taman-desa :  6
kepong :  6
gombak :  6
bangsar :  6
pudu :  6
kuchai-lama :  6
setapak :  6
brickfields :  6
cheras :  6
sungai-besi :  6
bandar-menjalara :  6
ampang :  6
taman-u-thant :  0
setapak :  6
setapak :  6
kenny-hills-(bukit-tunku) :  5
batu :  5
pudu :  5
bangsar :  5
ampang :  5
cheras :  5
bukit-bintang :  5
jinjang :  5
taman-u-thant :  0
pantai :  5
bangsar :  5
setapak :  5
pudu :  5
jalan-kuching :  5
bandar-tun-razak :  5
taman-u-thant :  0
ampang-hilir :  5
pudu :  5
setapak :  5
segambut :  5
brickfields :  5
sentul :  5
batu :  5
pudu :  5
sentul :  5
cheras :  4
sentul :  4
jalan-klang-lama-(old-klang-road) :  4
jalan-klang-lama-(old-klang-road) :  4
jalan-klang-lama-(old-klang-road) :  4
ampang :  4
setapak :  4
brickfields :  4
segambut :  4
damansara :  4
pudu :  4
bukit-bintang :  4
bangsar :  4
dutamas :  4
cheras :  4
taman-setiawangsa :  4
sungai-besi :  

jinjang :  2
damansara-heights :  2
keramat :  2
pudu :  2
kampung-sungai-penchala :  2
selayang :  2
taman-desa :  2
selayang :  2
bangsar :  2
bukit-bintang :  2
segambut :  2
brickfields :  2
bangsar :  2
kenny-hills-(bukit-tunku) :  2
setapak :  2
salak-selatan :  2
setapak :  2
jalan-klang-lama-(old-klang-road) :  2
brickfields :  2
pudu :  2
pudu :  2
jalan-klang-lama-(old-klang-road) :  2
dutamas :  2
ampang :  2
bangsar :  2
brickfields :  2
desa-pandan :  2
seputeh :  2
jalan-klang-lama-(old-klang-road) :  2
taman-u-thant :  0
bangsar :  1
brickfields :  1
bangsar :  1
bukit-bintang :  1
taman-u-thant :  0
seputeh :  1
jalan-klang-lama-(old-klang-road) :  1
pudu :  1
brickfields :  1
klcc :  1
setapak :  1
brickfields :  1
titiwangsa :  1
wangsa-maju :  1
cheras :  1
sungai-besi :  1
bandar-tun-razak :  1
salak-selatan :  1
kl-city :  1
taman-u-thant :  0
setapak :  1
cheras :  1
sentul :  1
segambut :  1
taman-u-thant :  0
ampang :  1
sentul :  1
wangsa-maju :  1
sentul :  1


# Modelling

## Preprocessing
To make use of both data, we can import the data and concat them. Some features could also be derived from the known data.

In [0]:
""" IMPORT LIBRARIES """
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline
import matplotlib.pyplot as plt

In [20]:
""" IMPORT DATA """
df_KL = pd.read_csv(pwd + 'output/kl.csv', sep = ';')
df_KL["region"] = "KL"
df_Selangor = pd.read_csv(pwd + 'output/selangor.csv', sep = ';')
df_Selangor["region"] = "Selangor"
df = pd.concat([df_KL, df_Selangor])

df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Drop duplicated rows or having missing values
df.dropna()
df.drop_duplicates(inplace=True)

# Calculate price per square foot (psf)
df["price_psf"] = df.price / df.area_sf

# Sort by address, area_sf and date 
df.sort_values(by=["addr", "area_sf", "date"], inplace=True, ascending=False)

# Remove rows having 1 transaction only
df = df[df.duplicated(subset=['addr', 'area_sf'], keep=False)]

print("No. of Rows (Raw data):", len(df.index))
df.head(10)

No. of Rows (Raw data): 234230


,addr,area_sf,date,house_type,price,area,neighbourhood,region,price_psf
125482,Zara Zero Lot Bunglow,3595.0,2013-05-31,Detached House,856518.0,bandar-country-homes,Saujana Rawang,Selangor,238.252573
125499,Zara Zero Lot Bunglow,3595.0,2012-07-27,Detached House,832098.0,bandar-country-homes,Saujana Rawang,Selangor,231.459805
125507,Zara Zero Lot Bunglow,3595.0,2012-03-20,Detached House,832098.0,bandar-country-homes,Saujana Rawang,Selangor,231.459805
125512,Zara Zero Lot Bunglow,3595.0,2012-02-28,Detached House,722184.0,bandar-country-homes,Saujana Rawang,Selangor,200.885675
185908,Wsl 09,1378.0,2013-07-11,Semi-Detached House,650000.0,kajang,Taman Saujana Suria,Selangor,471.698113
185915,Wsl 09,1378.0,2013-06-14,Semi-Detached House,650000.0,kajang,Taman Saujana Suria,Selangor,471.698113
185911,Wsl 02,1378.0,2013-06-28,Semi-Detached House,650000.0,kajang,Taman Saujana Suria,Selangor,471.698113
185913,Wsl 02,1378.0,2013-06-21,Semi-Detached House,650000.0,kajang,Taman Saujana Suria,Selangor,471.698113
185905,Wsd Suria Saujana,3498.0,2013-08-14,Semi-Detached House,928000.0,kajang,Taman Saujana Suria,Selangor,265.294454
185906,Wsd Suria Saujana,3498.0,2013-08-01,Semi-Detached House,928000.0,kajang,Taman Saujana Suria,Selangor,265.294454


A simple analytic using averaging will give us a result for linear regression on rate of psf change.

In [21]:
# Get min/max date and its price accordingly
df = df.groupby(["addr", "region", "area_sf", "house_type", "area", "neighbourhood"]).agg({"date": ["first", "last", "count"], "price_psf": ["first", "last"]})

# Calculate average change of price psf by day
df["price_psf_change"] = df.price_psf["first"] - df.price_psf["last"]
df["number_of_days"] = (pd.to_datetime(df.date["first"]) - pd.to_datetime(df.date["last"])).apply(lambda x: x.days)
df.drop(df[df["number_of_days"] == 0].index, inplace=True)
df["avg_price_psf_change"] = df["price_psf_change"] / df["number_of_days"]


df["transaction_count"] = df.date['count']
df.drop(df[df["transaction_count"] < 3].index, inplace=True)

# Drop unnecessary columns
df.reset_index(inplace=True)
df.drop(["addr", "date", "price_psf", "price_psf_change"], axis=1, inplace=True, level=0)

print("No. of Rows (Processed data):", len(df.index))
# df.head(10)
df.sort_values('avg_price_psf_change', ascending=False).head(10)

No. of Rows (Processed data): 23267


,region,area_sf,house_type,area,neighbourhood,number_of_days,avg_price_psf_change,transaction_count
,,,,,,,,
23172,Selangor,786.0,Condominium/Apartment,sunway-damansara,Sunway Sutera Condominium,21,13.934327,3
20453,Selangor,1399.0,Terrace House,klang,Taman Mewah Jaya,5,12.151537,3
12846,Selangor,850.0,Flat,mutiara-damansara,Surian Condominium,174,9.228736,3
4676,Selangor,11.0,Land,bangi,Bandar Bukit Mahkota Bangi,137,6.635700,3
21868,Selangor,452.0,Condominium/Apartment,cyberjaya,Garden Plaza,29,5.340250,3
21222,KL,1130.0,Condominium/Apartment,kl-city,Putra Court,25,5.309735,3
14677,Selangor,1647.0,Terrace House,puchong-south,Bandar Sierra Puchong,34,3.948355,3
1238,Selangor,980.0,Condominium/Apartment,petaling-jaya,Li Villas,172,3.915520,3
6958,KL,2680.0,Condominium/Apartment,taman-desa,Casa Desa,47,3.810734,3


Some encoding to make them enumerable, this is useful for training.

In [22]:
# filtering landed/non-landed
non_landed = ['Flat', 'Condominium/Apartment', 'Hotel/Service Apartment']
df['is_landed_property'] = [1 if d in non_landed else 0 for d in df['house_type']]

# Encode categorical columns
le_r = preprocessing.LabelEncoder()
le_h = preprocessing.LabelEncoder()
le_a = preprocessing.LabelEncoder()
le_n = preprocessing.LabelEncoder()
df["region_id"] = le_r.fit_transform(df["region"])
df["house_type_id"] = le_h.fit_transform(df["house_type"])
df["area_id"] = le_a.fit_transform(df["area"])
df["neighbourhood_id"] = le_n.fit_transform(df["neighbourhood"])
df.head() # aftermath showing

,region,area_sf,house_type,area,neighbourhood,number_of_days,avg_price_psf_change,transaction_count,is_landed_property,region_id,house_type_id,area_id,neighbourhood_id
,,,,,,,,,,,,,
0,Selangor,1647.0,Terrace House,shah-alam,"Seksyen 7, Shah Alam",140,-0.145719,4,0,1,9,104,1778
1,Selangor,958.0,Cluster House,puchong,Taman Tasik Puchong,1810,0.083623,7,0,1,0,86,2774
2,Selangor,958.0,Cluster House,puchong,Taman Tasik Puchong,1316,0.099149,3,0,1,0,86,2774
3,Selangor,926.0,Condominium/Apartment,shah-alam,Alam Budiman,201,0.128945,6,1,1,1,104,43
4,Selangor,1173.0,Terrace House,shah-alam,"Section 33, Shah Alam",111,-0.039400,3,0,1,9,104,1743


Shuffling data

In [31]:
df2 = shuffle(df)
# X_raw = df[["region_id", "house_type_id", "area_id", "neighbourhood_id", "area_sf"]]
X_raw = df2[["region_id", "house_type_id", "area_id", "neighbourhood_id", "area_sf", "is_landed_property", "number_of_days"]]
y = df2["avg_price_psf_change"]


# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_raw)
# X=scaler.transform(X_raw)
X=X_raw
X.head(10)
# X

,region_id,house_type_id,area_id,neighbourhood_id,area_sf,is_landed_property,number_of_days
,,,,,,,
8497,0,1,34,1575,797.0,1,1685
21146,1,3,25,2182,517.0,1,155
19153,1,9,127,2930,1647.0,0,849
20622,1,9,70,2629,1130.0,0,2130
2958,1,9,92,260,1604.0,0,552
17372,1,9,104,2709,1399.0,0,604
9926,1,9,87,2396,1539.0,0,1815
8700,0,3,83,617,861.0,1,1551
15292,1,1,102,1824,818.0,1,1744


## Linear Regression

### Model Training and Prediction

In [32]:
""" LINEAR REGRESSION """
       
# Split training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
       
linearReg = LinearRegression()
y_pred = linearReg.fit(X_train, y_train).predict(X_test)
# Root mean squared error
print("Root mean square error for test dataset: {}".format(np.round(np.sqrt(mean_squared_error(y_test, y_pred)), 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Root mean square error for test dataset: 0.32
Variance score: -0.04


### Cross Validation for Linear Regression

In [33]:
""" CROSS VALIDATION """
kfold = KFold(n_splits=10)
modelCV = LinearRegression()
scoring = "neg_mean_squared_error"
results = cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average RMSE: %.3f" % np.sqrt(np.abs(results.mean())))

10-fold cross validation average RMSE: 1.759


### Result of prediction
Since we are targetting the highest return, we should sort it based on `y_pred`.

First, merge the prediction dataframe with the test data:

In [34]:
test = pd.DataFrame(data=(X_test))
test['raise_per_day_prediction'] = y_pred
# test = test.sort_values('raise_per_day_prediction', ascending=False)
test.head()

,region_id,house_type_id,area_id,neighbourhood_id,area_sf,is_landed_property,number_of_days,raise_per_day_prediction
,,,,,,,,
14961,0,1,97,2879,1109.0,1,1691,0.039955
6919,1,1,100,2231,818.0,1,273,0.042487
14841,0,4,39,5,1001.0,1,333,0.008889
17504,1,9,15,1651,1647.0,0,398,0.020989
13564,1,1,86,3021,969.0,1,1729,0.120279


Then transform it into readable table with relevant data and sort it by the estimated return.

In [48]:
prediction = pd.DataFrame(data={})

prediction['State'] = [le_r.inverse_transform(k) for k in test['region_id']]
prediction['Area'] = [le_a.inverse_transform(k) for k in test['area_id']]
prediction['Neighbourhood or Township'] = [le_n.inverse_transform(k) for k in test['neighbourhood_id']]

prediction['Property Type'] = [le_h.inverse_transform(k) for k in test['house_type_id']]
prediction['Area in sq. ft'] = test['area_sf']
prediction['Duration of Investment (days)'] = test['number_of_days']
prediction['Prediction on Price Appreciation per day(per sf)'] = test['raise_per_day_prediction']

prediction['Price Appreciation per day'] = [ ("RM%.2f" % k) for k in test['raise_per_day_prediction']*test['area_sf']]

prediction['er'] = test['raise_per_day_prediction'] * test['number_of_days'] * test['area_sf']

# sort by er
prediction = prediction.sort_values('er', ascending=False).reset_index()

# beautify er and drop
prediction['Estimated return'] = [ "RM{0:,.2f}".format(k) for k in prediction['er']]
prediction.drop(columns=['er', 'index'], inplace=True)
prediction.head(10)



,State,Area,Neighbourhood or Township,Property Type,Area in sq. ft,Duration of Investment (days),Prediction on Price Appreciation per day(per sf),Price Appreciation per day,Estimated return
0,Selangor,pandan-indah,Taman Pandan Indah,Flat,32701.0,522.0,0.272755,RM90.93,"RM4,655,914.84"
1,KL,dutamas,Solaris Dutamas,Hotel/Service Apartment,21776.0,426.0,0.130606,RM41.40,"RM1,211,575.15"
2,Selangor,damansara-perdana,Armanee Terrace,Condominium/Apartment,2400.0,2059.0,0.124481,RM85.77,"RM615,136.54"
3,Selangor,bandar-damai-perdana,Bandar Damai Perdana,Terrace House,2799.0,1858.0,0.109938,RM75.24,"RM571,736.51"
4,Selangor,shah-alam,"Section 13, Shah Alam",Terrace House,2788.0,1583.0,0.129285,RM37.74,"RM570,589.14"
5,Selangor,bukit-rahman-putra,Taman Bukit Rahman Putra,Semi-Detached House,4499.0,1467.0,0.083981,RM52.37,"RM554,279.16"
6,Selangor,puchong,Solace Service Apartment @ Setiawalk,Hotel/Service Apartment,2142.0,1847.0,0.138961,RM158.73,"RM549,765.99"
7,KL,wangsa-maju,Villa Wangsamas,Condominium/Apartment,4402.0,1466.0,0.081418,RM117.93,"RM525,415.17"
8,Selangor,rawang,Desa Puteri,Terrace House,2013.0,2016.0,0.118018,RM45.79,"RM478,941.74"
9,Selangor,usj-heights,USJ Heights,Terrace House,2239.0,1939.0,0.109069,RM103.99,"RM473,515.26"
